In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import cv2
import os
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [2]:
import os
os.getcwd() #to get current working directory

'/Users/ashishsingh/AIML/Assignments/Introduction to Neural Networks and Deep Learning_R7_INN_Project2'

Understand the basic Image Classification pipeline and the data-driven approach (train/predict stages)

In [3]:
import h5py
filename = 'SVHN_single_grey1.h5'
dataframe = h5py.File(filename, 'r')


In [4]:
# # List all groups
print("Keys: %s" % dataframe.keys())

Keys: <KeysViewHDF5 ['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']>
X_test


In [19]:
datasetnames = [n for n in dataframe.keys()]

In [21]:
for n in datasetnames:
    print(n) 

X_test
X_train
X_val
y_test
y_train
y_val


In [ ]:
x_test = dataFrame['X_test']
x_train = dataFrame['X_train']
x_val = dataFrame['X_val']
y_test = dataFrame['y_test']
y_train = dataFrame['y_train']
y_val = dataFrame['y_val']

In [ ]:
for i in dataframe.keys():
    print(dataframe[i])

In [5]:
dataframe['X_train']

<HDF5 dataset "X_train": shape (42000, 32, 32), type "<f4">

In [6]:
len(list(dataframe['X_train']))

42000

In [7]:
len(list(dataframe['X_train'])[0])

32

In [8]:
list(dataframe['X_train'])[0]

array([[ 33.0704,  30.2601,  26.852 , ...,  71.4471,  58.2204,  42.9939],
       [ 25.2283,  25.5533,  29.9765, ..., 113.0209, 103.3639,  84.2949],
       [ 26.2775,  22.6137,  40.4763, ..., 113.3028, 121.775 , 115.4228],
       ...,
       [ 28.5502,  36.212 ,  45.0801, ...,  24.1359,  25.0927,  26.0603],
       [ 38.4352,  26.4733,  23.2717, ...,  28.1094,  29.4683,  30.0661],
       [ 50.2984,  26.0773,  24.0389, ...,  49.6682,  50.853 ,  53.0377]],
      dtype=float32)

# Implement and apply an optimal k-Nearest Neighbor (kNN) classifier

In [15]:
len(cv2.resize(list(dataframe['X_train'])[0], (32,32)).flatten())

1024

In [ ]:
# flattening from 3D to 2D
X_train = np.reshape(x_train,(42000,32*32))

In [ ]:
NNH = KNeighborsClassifier(n_neighbors=10)

In [ ]:
NNH.fit(X_train,y_train)

In [ ]:
# predict the model
# Change the dimension from 3D -> 2D
X_test = np.reshape(x_test,(18000,32*32))
prediction = NNH.predict(X_test)

In [ ]:
#Accuracy
print("Accuracy:", metrics.accuracy_score(prediction,y_test))

In [ ]:
#Confusion matrix
print("Confusion Matrix:", metrics.confusion_matrix(prediction,y_test))

In [ ]:
#Classification report
cr=metrics.classification_report(y_test,prediction)
print(cr)

### Implement and apply a deep neural network classifier including (feedforward neural network, RELU activations)

In [ ]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

In [ ]:
#Reshape data from 2D to 1D -> 32X32 to 1024
model.add(tf.keras.layers.Reshape((1024,),input_shape=(32,32)))

### Implement batch normalization for training the neural network

In [ ]:
#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

In [ ]:
model.add(tf.keras.layers.Dense(150, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))

In [ ]:
#Output layer
model.add(tf.keras.layers.Dense(10, activation='softmax', name='Output'))

### Understand and be able to implement (vectorized) backpropagation (cost stochastic gradient descent, cross entropy loss, cost functions)

In [ ]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Understand the differences and trade-offs between traditional and NN classifiers with the help of classification metrics

In [ ]:
# Change train and test labels into one-hot vectors
trainY = tf.keras.utils.to_categorical(y_train, num_classes=10)
testY = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
#Train the model
features_val_arr = np.array(x_test)
model.fit(x_train,trainY,          
          validation_data=(features_val_arr, testY),
          epochs=75,
          batch_size=150,validation_split = 0.01,
         shuffle='batch')

In [ ]:
# model evaluation
evaluate = model.evaluate(x_text, testY)
print(evaluate)

In [ ]:
# predict the model
y_predict=model.predict_classes(x_text)

In [ ]:
cr=metrics.classification_report(y_test,y_predict)
print(cr)

### Understand the differences and trade-offs between traditional and NN classifiers with the help of classification metrics

In [16]:
def image_to_feature_vector(image, size=(32, 32)):
	# resize the image to a fixed size, then flatten the image into
	# a list of raw pixel intensities
	return cv2.resize(image, size).flatten()

In [ ]:
# # train and evaluate a k-NN classifer on the histogram
# # representations
# print("[INFO] evaluating accuracy...")

# NNH.fit(X_train_new, y_train)
# acc = NNH.score(X_train_new, y_train)
# print("[INFO]  accuracy: {:.2f}%".format(acc * 100))

In [ ]:
#classification metrics report
# sklearn.metrics.classification_report(y_test, y_pred, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False)

In [ ]:
# # K fold
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score

# # creating odd list of K for KNN
# myList = list(range(1,50))


# # empty list that will hold cv scores
# cv_scores = []
# k_neighbors = []

# # perform 10-fold cross validation
# for k in myList:
#     knn = KNeighborsClassifier(n_neighbors=k)
#     scores = cross_val_score(knn, X_train_new, y_train, cv=10, scoring='accuracy')
#     cv_scores.append(scores.mean())
#     k_neighbors.append(k)


# MSE = [1 - x for x in cv_scores]
# min(MSE)
# MSE.index(min(MSE))
# best_k = myList[MSE.index(min(MSE))]
# print ("The optimal number of neighbors is %d" % best_k)
